Data
- Water customer locations (address, customer type, spatial coordinates, etc). Probably in CSV format.
- Parcel boundaries for residential parcels with landscape measurements attached (sq.ft. of irrigated and irrigable area, etc) - shapefile
- Parcel boundaries for all parcels without landscape measurements - shapefile
- Water use data is available, but likely not needed for this project. Might be better for us to fake some data.

Potential functionality Baseline:
- Display parcels on a map
- Display customers on a map
- Display basic information about customers, especially for selected customer
- Display on a Google maps (or similar) satellite basemap. Or maybe a stylized basemap when zoomed out and a photo basemap when zoomed in?
- Play with visualization and visual aesthetic - could help to involve our designer here to give feedback on different approaches
do we want customer = parcel? or customers as points and parcels as polygons? 
- What about multiple customers on one parcel? 
- How to highlight currently selected customer?
- How to show off different attributes of the customers, such as volume of water use or level of efficiency? Colored points? colored parcel boundaries?
- Play with showing different amounts or granularities of data at different zoom levels

More advanced if time and interest allow:
- User can draw an arbitrary polygon on the map and get back a measurement of how much area is enclosed within.
- Ability to drag points around the map, e.g. if a point is in the wrong location user could correct its location
- Alternately, ability to associate a customer with a parcel(s) - e.g. click customer then click parcel to associate them.

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from shapely import wkt

# import data
data = pd.read_csv('data/cust_loc_mnwd.csv')

# make new dataframe with desired columns
data_points = data[['geom','cust_loc_water_type', 'cust_loc_class']].dropna()
data_points['geom'] = data_points['geom'].apply(wkt.loads)

# reproject data to geodataframe and set crs = 4326
gdf_out = gpd.GeoDataFrame(data_points, geometry='geom')
gdf_out = gdf_out.set_crs(4326)

/tmp/ipykernel_23175/3764432507.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/cust_loc_mnwd.csv')


In [9]:
gdf_out['lon'] = gdf_out.geom.x
gdf_out['lat'] = gdf_out.geom.y

In [3]:
#type(data)

In [4]:
#data

In [5]:
# read parcel data and convert to geodataframe
parcels = gpd.read_file('data/Parcels_Queried.shp')

parcels_out = gpd.GeoDataFrame(parcels, geometry='geometry')

ERROR 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


In [6]:
#parcels.head()

In [7]:
# reproject data to crs = 4326
parcels_reprojected = parcels_out.to_crs(4326)

In [8]:
# select important columns
parcels_col = parcels_reprojected[['APN', 'geometry']]

In [10]:
# spacially join gdf_out and parcels
gdf_parcels = gpd.sjoin(gdf_out, parcels_col)

In [23]:
# this is the map that runs but it does not produce the tooltip

import folium
from folium.plugins import FastMarkerCluster
m = folium.Map([33.580039,-117.757444],
                  zoom_start=12,
                  tiles='cartodbpositron')
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
folium.features.GeoJson(parcels_reprojected).add_to(m)

m.add_child(FastMarkerCluster(gdf_parcels[['lat', 'lon', 'APN']].values.tolist()))
m.save("water_map_2.html")


In [11]:
# create dataframe with desired columns for tooltip
test = gdf_out[['geom','cust_loc_water_type']]

In [12]:
# this map does not run because it is too big, but if that were not an issue it would produce the tooltip

import folium
from folium.plugins import FastMarkerCluster

m = folium.Map([33.580039,-117.757444],
                  zoom_start=12,
                  tiles='cartodbpositron')

folium.features.GeoJson(parcels_reprojected).add_to(m)
#GeoJson(text).add_to(m)

m.add_child(FastMarkerCluster(gdf_parcels[['lat', 'lon', 'APN']].values.tolist()))
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
#mapa.add_children(points)

folium.features.GeoJson(test,  
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['cust_loc_water_type'],
                                                                aliases = ['Water Type'],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(m)
m.save("water_map1.html")
